In [26]:
import keras
import numpy as np
from sklearn.metrics import classification_report
from sklearn.datasets import load_iris
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler

In [27]:
X, y = load_iris(return_X_y=True)
xtrainF, xtestF, ytrainF, ytestF = train_test_split(X, y, random_state=42 ,test_size=0.20)
xtrainF.shape

(120, 4)

In [28]:
pesos = compute_class_weight(class_weight='balanced',classes=np.unique(ytrainF),y=ytrainF)
pesos = {
    0:pesos[0],
    1:pesos[1]
}

In [29]:
def Model():
    initalizaer = keras.initializers.VarianceScaling(scale=2., mode='fan_in', distribution='normal',seed=32)

    model = keras.models.Sequential(
        layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Dense(8, activation=keras.activations.relu, kernel_initializer=initalizaer),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(4, activation=keras.activations.relu, kernel_initializer=initalizaer),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(3, activation=keras.activations.softmax)
        ]
    )
    return model

## Momentum

In [30]:
model = Model()
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32


24/24 [==============================] - 1s 2ms/step - loss: 1.4531 - sparse_categorical_accuracy: 0.4000
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 0.9505 - sparse_categorical_accuracy: 0.5917
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8527 - sparse_categorical_accuracy: 0.6333
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7892 - sparse_categorical_accuracy: 0.6667
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7879 - sparse_categorical_accuracy: 0.6333
Epoch 6/32
24/24 [==============================] - 0s 2ms/step - loss: 0.6915 - sparse_categorical_accuracy: 0.6750
Epoch 7/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7016 - sparse_categorical_accuracy: 0.6750
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7740 - sparse_categorical_accuracy: 0.6500
Epoch 9/32
24/24 [==============================] - 0s 2ms/step - loss: 0.6

1/1 [==============================] - 0s 141ms/step


In [32]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [33]:
model.save('model/Model_iris0.keras')
model = keras.models.load_model('model/Model_iris0.keras')
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, verbose=0)

In [34]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 134ms/step


In [35]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



&nbsp;

## Nesterov

In [36]:
model = Model()
model.compile(
    optimizer = keras.optimizers.SGD(learning_rate=0.001,momentum=0.9, nesterov=True),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32


24/24 [==============================] - 1s 2ms/step - loss: 1.0737 - sparse_categorical_accuracy: 0.4667
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8842 - sparse_categorical_accuracy: 0.6583
Epoch 3/32
24/24 [==============================] - 0s 1ms/step - loss: 0.8449 - sparse_categorical_accuracy: 0.7000
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8175 - sparse_categorical_accuracy: 0.6583
Epoch 5/32
24/24 [==============================] - 0s 1ms/step - loss: 0.7172 - sparse_categorical_accuracy: 0.7417
Epoch 6/32
24/24 [==============================] - 0s 2ms/step - loss: 0.6333 - sparse_categorical_accuracy: 0.8000
Epoch 7/32
24/24 [==============================] - 0s 2ms/step - loss: 0.6882 - sparse_categorical_accuracy: 0.7500
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 0.5686 - sparse_categorical_accuracy: 0.8333
Epoch 9/32
24/24 [==============================] - 0s 2ms/step - loss: 0.6

In [37]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 131ms/step


In [38]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



### Adagrad

In [39]:
model = Model()
model.compile(
    optimizer = keras.optimizers.Adagrad(),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32


24/24 [==============================] - 1s 2ms/step - loss: 1.4875 - sparse_categorical_accuracy: 0.1667
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4212 - sparse_categorical_accuracy: 0.2083
Epoch 3/32
24/24 [==============================] - 0s 1ms/step - loss: 1.3978 - sparse_categorical_accuracy: 0.2583
Epoch 4/32
24/24 [==============================] - 0s 1ms/step - loss: 1.3673 - sparse_categorical_accuracy: 0.1917
Epoch 5/32
24/24 [==============================] - 0s 1ms/step - loss: 1.3196 - sparse_categorical_accuracy: 0.2333
Epoch 6/32
24/24 [==============================] - 0s 1ms/step - loss: 1.3834 - sparse_categorical_accuracy: 0.2000
Epoch 7/32
24/24 [==============================] - 0s 1ms/step - loss: 1.2858 - sparse_categorical_accuracy: 0.3250
Epoch 8/32
24/24 [==============================] - 0s 1ms/step - loss: 1.3374 - sparse_categorical_accuracy: 0.2417
Epoch 9/32
24/24 [==============================] - 0s 1ms/step - loss: 1.2

In [40]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 133ms/step


In [41]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       0.19      0.30      0.23        10
           1       0.33      0.11      0.17         9
           2       0.27      0.27      0.27        11

    accuracy                           0.23        30
   macro avg       0.26      0.23      0.22        30
weighted avg       0.26      0.23      0.23        30



&nbsp;

### Adam

In [47]:
model = Model()
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32
24/24 [==============================] - 1s 2ms/step - loss: 1.6009 - sparse_categorical_accuracy: 0.2667
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 1.4685 - sparse_categorical_accuracy: 0.2333
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 1.2996 - sparse_categorical_accuracy: 0.2833
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 1.2733 - sparse_categorical_accuracy: 0.3417
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 1.2017 - sparse_categorical_accuracy: 0.3333
Epoch 6/32
24/24 [==============================] - 0s 3ms/step - loss: 1.0734 - sparse_categorical_accuracy: 0.4583
Epoch 7/32
24/24 [==============================] - 0s 2ms/step - loss: 1.0035 - sparse_categorical_accuracy: 0.5917
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 0.9857 - sparse_categorical_accuracy: 0.5833
Epoch 9/32
24/24 [==============================] - 0s 2ms/step 

In [48]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 152ms/step


In [49]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [51]:
keras.backend.epsilon()

1e-07

### Nadam

In [53]:
model = Model()
model.compile(
    optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    loss      = keras.losses.sparse_categorical_crossentropy,
    metrics   = [keras.metrics.sparse_categorical_accuracy]
)
call = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model.fit( xtrainF, ytrainF, epochs=32, batch_size=5, callbacks=[call])

Epoch 1/32
24/24 [==============================] - 2s 2ms/step - loss: 1.0133 - sparse_categorical_accuracy: 0.4667
Epoch 2/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8998 - sparse_categorical_accuracy: 0.5667
Epoch 3/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8243 - sparse_categorical_accuracy: 0.6167
Epoch 4/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8473 - sparse_categorical_accuracy: 0.6167
Epoch 5/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8131 - sparse_categorical_accuracy: 0.6083
Epoch 6/32
24/24 [==============================] - 0s 2ms/step - loss: 0.8198 - sparse_categorical_accuracy: 0.6167
Epoch 7/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7364 - sparse_categorical_accuracy: 0.6500
Epoch 8/32
24/24 [==============================] - 0s 2ms/step - loss: 0.7465 - sparse_categorical_accuracy: 0.6417
Epoch 9/32
24/24 [==============================] - 0s 2ms/step 

In [55]:
pred = np.argmax(model.predict(xtestF),axis=-1)

1/1 [==============================] - 0s 127ms/step


In [56]:
print(classification_report(ytestF,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30

